In [23]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
import re
import json
from datetime import datetime

### Download

In [24]:
url = 'http://publichealth.lacounty.gov/media/monkeypox/maps/first/mpx-spamap-widget.html'

In [25]:
page = requests.get(url, verify=False)

### Parse

In [26]:
soup = BeautifulSoup(page.content, "html.parser")

In [27]:
for elem in soup(text=re.compile(r"Case Counts")):
    script_content = elem

In [28]:
data = json.loads(script_content)

In [7]:
regions_list = data["x"]["calls"][1]["args"][4]

In [98]:
for item in data['x']['calls']:
    print(item['method'])

addProviderTiles
addPolygons
addPolygons
addMarkers
addLegend
addMarkers
addControl
addLayersControl


In [117]:
lats = data['x']['calls'][5]['args'][0]

In [118]:
longs = data['x']['calls'][5]['args'][1]

In [125]:
indeces = data['x']['calls'][5]['args'][2]['iconUrl']['index']

In [145]:
dict_list = []
for idx, lat in enumerate(lats):
    d = dict(
        lat = lat,
        long = longs[idx],
        symbol = indeces[idx],
    )
    dict_list.append(d)

In [146]:
pd.DataFrame(dict_list)

,lat,long,symbol
0,34.036970,-118.265280,0
1,34.101943,-118.296456,0
2,34.103816,-118.309471,0
3,34.160435,-118.448495,0
4,34.067381,-118.376628,0
...,...,...,...
97,33.881479,-118.288525,0
98,33.949231,-118.352975,0
99,34.073439,-118.379923,0
100,34.076216,-118.373378,0


In [8]:
dict_list = []
TAG_RE = re.compile(r"<[^>]+>")
for z in regions_list:
    split = z.split("<br>")
    area = TAG_RE.sub("", str(split[0]))
    spa = area[:7]
    region_name = re.split('[0-9]', area)[1].strip()
    cases = TAG_RE.sub("", str(split[1]))
    cases_clean = cases.split(':')[1].strip()
    
    d = dict(
        spa=spa,
        region=region_name,
        confirmed_cases=cases_clean
    )
    dict_list.append(d)

In [9]:
df = pd.DataFrame(dict_list)

### Get date

In [10]:
dashboard_url = 'http://publichealth.lacounty.gov/media/monkeypox/data/index.htm'

In [11]:
dash_page = requests.get(dashboard_url, verify=False)

In [12]:
dash_soup = BeautifulSoup(dash_page.content, "html.parser")

In [13]:
content = dash_soup.find("div", id="content")

In [14]:
for elem in content(text=re.compile(r"Updated on Fridays. Data as of")):
    date_text = elem

In [15]:
date_str = date_text.strip("Updated on Fridays. Data as of ")
date_str

'August 5, 2022'

In [16]:
date = pd.to_datetime(date_str).date()

In [17]:
df['date'] = date

In [18]:
df["confirmed_cases"] = df["confirmed_cases"].astype(int)

In [19]:
df

,spa,region,confirmed_cases,date
0,SPA 1,Antelope Valley,8,2022-08-05
1,SPA 2,San Fernando,76,2022-08-05
2,SPA 3,San Gabriel,17,2022-08-05
3,SPA 4,Metro,259,2022-08-05
4,SPA 5,West,27,2022-08-05
5,SPA 6,South,40,2022-08-05
6,SPA 7,East,37,2022-08-05
7,SPA 8,South Bay,31,2022-08-05


### Export date-stamped CSV

In [20]:
df.to_csv(f"raw/los-angeles/{date}.csv", index=False)

In [22]:
df.to_csv(f"raw/los-angeles/latest.csv", index=False)